!pip install gluonnlp==0.9.1

!pip install sentencepiece==0.1.85
- 해당버전으로 설치하지 않으면 tokenizer() 실행 시, nbest_size 런타임에러 남!!

In [1]:
import os

import numpy as np
import tensorflow as tf 

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

In [2]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]
    
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [3]:
nlp.__version__

'0.9.1'

In [4]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

# from gluonnlp.data import SentencepieceTokenizer
tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
#                                   num_best=0, # 디폴트
#                                   alpha=1.0) # 디폴트
"""
num_best(int): A scalar for sampling subwords.
- 0,1: 샘플링 미실시
- 2 이상: 샘플링 실시 
- -1 이하: 모든 후보에서 샘플링
alpha(float): A scalar for a smoothing parameter.
"""

# import gluonnlp as nlp
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                              mask_token=None,
                                              sep_token=None,
                                              cls_token=None,
                                              unknown_token='<unk>',
                                              padding_token='<pad>',
                                              bos_token='<s>',
                                              eos_token='</s>')

In [5]:
vocab['안녕']

28911

In [6]:
vocab['▁나는']

3918

In [7]:
a = "안녕하세요 나는 사람입니다."
b = tokenizer(a)
b

['▁안녕하세요', '▁나는', '▁사람', '입니다', '.']

In [8]:
nlp.vocab

<module 'gluonnlp.vocab' from 'C:\\Users\\answl\\anaconda3\\envs\\nlp\\lib\\site-packages\\gluonnlp\\vocab\\__init__.py'>

In [9]:
print(nlp.vocab)

<module 'gluonnlp.vocab' from 'C:\\Users\\answl\\anaconda3\\envs\\nlp\\lib\\site-packages\\gluonnlp\\vocab\\__init__.py'>


In [10]:
help(nlp.vocab)

Help on package gluonnlp.vocab in gluonnlp:

NAME
    gluonnlp.vocab - Vocabulary.

PACKAGE CONTENTS
    bert
    elmo
    subwords
    vocab

CLASSES
    builtins.object
        gluonnlp.vocab.elmo.ELMoCharVocab
        gluonnlp.vocab.subwords.SubwordFunction
            gluonnlp.vocab.subwords.ByteSubwords
            gluonnlp.vocab.subwords.NGramHashes
        gluonnlp.vocab.vocab.Vocab
            gluonnlp.vocab.bert.BERTVocab
    
    class BERTVocab(gluonnlp.vocab.vocab.Vocab)
     |  BERTVocab(counter=None, max_size=None, min_freq=1, unknown_token='[UNK]', padding_token='[PAD]', bos_token=None, eos_token=None, mask_token='[MASK]', sep_token='[SEP]', cls_token='[CLS]', reserved_tokens=None, token_to_idx=None)
     |  
     |  Specialization of gluonnlp.Vocab for BERT models.
     |  
     |  BERTVocab changes default token representations of unknown and other
     |  special tokens of gluonnlp.Vocab and adds convenience parameters to specify
     |  mask, sep and cls tokens typic

In [15]:
nlp.vocab.BERTVocab

gluonnlp.vocab.bert.BERTVocab

In [16]:
nlp.vocab.BERTVocab()

Vocab(size=5, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [17]:
print(nlp.vocab.BERTVocab)

<class 'gluonnlp.vocab.bert.BERTVocab'>


In [11]:
vocab

Vocab(size=50000, unk="<unk>", reserved="['<pad>', '<s>', '</s>']")

In [12]:
print(vocab)

Vocab(size=50000, unk="<unk>", reserved="['<pad>', '<s>', '</s>']")


In [9]:
def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word # 문장 생성 시작 단어
    toked = tokenizer(sent) # 해당 단어 토크나이즈 
    # SentencepieceTokenizer("토크나이저 경로")
    # ex. tokenizer("이때") --> ['▁이때']
    
    for _ in range(max_step): # 100번 반복
        input_ids = tf.constant([vocab[vocab.bos_token],] + vocab[toked])[None, :]
        # vocab[vocab.bos_token] = vocab['<s>'] = 0
        # ex. vocab[toked] = vocab[tokenizer("이때")] = vocab[['▁이때']] = [4499]
        # [0,] + [4499] --> [0, 4499]
        # tf.constant([0, 4499]) --> <tf.Tensor: shape=(2,), dtype=int32, numpy=array([   0, 4499])>
        # array([   0, 4499])[None, :] --> array([[   0, 4499]]) # 차원 확장
        
        # (2)
        # vocab[['▁이때', '문에']]--> [4499, 2592]
        # input_ids: [0, 4499, 2592]--> array([[0, 4499, 2592]])
        
        outputs = model(input_ids)[:,-1,:]
        # GPT2Model([[0, 4499]])
        # 마지막 토큰의 은닉상태행렬: (1,50000) 크기
        
        if greedy: # (1) 단순히 가장 확률이 높은 단어를 선택하기
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
            # tf.argmax([[-1.33..., 2.33,..., ...]]) --> array([0, 0, 0, ...])
            # tf.argmax([[-1.33..., 2.33...., ...]], axis=-1) --> <tf.Tensor: ... array([2592])...>
            # tf.argmax(__).numpy() --> array([2592])
            # tf.argmax(__).numpy().tolist() --> [2592]
            # tf.argmax(__).numpy().tolist()[0] --> 2592
            # vocab.to_tokens(2592) --> '문에'
            
        else:      # (2) 확률이 높은 단어들 중 확률 분포에 따라 선택하기 
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
            # tf.random.categorical([[2592]], 1)[0] --> [2592]
            # vocab.to_tokens([2592]) --> ['문에'] --> [0] --> '문에'
            # gen = '문에'
        
        # 문장 마지막('</s>')을 예측한다면 예측 반복 종료! 
        if gen == '</s>':
            break
        
        sent += gen.replace('▁', ' ')
        # 밑줄 표시를 띄어쓰기로 변경 후
        # 원래 sent (생성 시작 단어)에 연결
        toked = tokenizer(sent)
        # 그리고 toked 새롭게 정의 
        # ex. tokenizer("이때문에 ")
        # --> ['▁이때', '문에']
    
    return sent

def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy() # array([1.33, 2.22, ....])
                             # TensorShape([50000])
    top_k = min(top_k, logits.shape[-1])  # min(top_k, 50000)
    
    if top_k > 0: # 상위 k개 랭크를 고른다고 한다면 
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        # tf.math.top_k(outputs[0], 3)--> TopKV2(values=<tf.Tensor:...> , indices=<tf.Tensor:...>)
        # ___[0]: <tf.Tensor: ... numpy=array([8.84.., 8.14.., 7.45...])...>
        # ___[..., -1, None]: <...[7.45...]...>
        # indices_to_remove : <... [True, True, True, ...] ...>
        _logits[indices_to_remove] = filter_value
        # array([-1.13.., 2.88.., ...]) --> 필터될 애들들
        # 얘네들에 필터값 -99999 대입 --> _logits 베열 수정됨 
        # _logits: array([-99999., -99999., -99999., .....])

    if top_p > 0.0: # 특정 확률 이상의 것들로만 필터링한다고 한다면
        # 1) 값 내림차순 정렬: 축적확률 보기 위해 
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        # tf.Tensor 타입 내림차순 정렬!! 
        # ex. ...array([  8.840773 ,   8.142104 ,   7.456431 , ... ])...
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        # ex. ...array([ 2592, 47441, 47453, ...)...
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)
        # tf.nn.softmax(sorted_logits, axis=-1): array([6.9024004e-02, 3.4321975e-02, 1.7289728e-02, ...]) --> 소프트맥스 확률 변환
        # tf.math.cumsum(__, axis=-1) : 축적 확률 - array([0.069024  , 0.10334598, 0.1206357 , ...
        
        sorted_indices_to_remove = cumulative_probs > top_p
        # 특정 확률(ex. 0.20) 넘어간다면(축적확률이 더 크다면) True 처리 --> 필터링할(버릴) 값
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        # __[..., :-1]: 마지막 필터링 여부값(True) 버림
        # tf.concat([[False], [True, True, ...]], axis=0) --> [False, True, True, ...]
        # 첫 값은 꼭 필터링 False --> 남김 !! 
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        # 제거할 인덱스들: array([18656,   512, 47462, ...,    73, 42568,     3])
        _logits[indices_to_remove] = filter_value
        # 해당 인덱스들에 필터값 -99999 대입 
        # array([생존1, 생존2, ... , -99999, -99999])
    
    return tf.constant([_logits])
    # array([[-99999., -99999., ..., 8.84077262878418, ... 8.142104148864746, ..., 7.456430912017822, ... -99999]])

In [111]:
vocab[['▁이때', '문에']]

[4499, 2592]

In [109]:
tokenizer('이때문에')

['▁이때', '문에']

In [104]:
tf.argsort(outputs[0], direction='DESCENDING')[tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)]

<tf.Tensor: shape=(49990,), dtype=int32, numpy=array([18656,   512, 47462, ...,    73, 42568,     3])>

In [103]:
tf.concat([[False], [True, True]], axis=0)

<tf.Tensor: shape=(3,), dtype=bool, numpy=array([False,  True,  True])>

In [99]:
sorted_indices_to_remove = tf.math.cumsum(tf.nn.softmax(tf.sort(outputs[0], direction='DESCENDING'), axis=-1), axis=-1)>0.20
sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[...,:-1]], axis=0)
sorted_indices_to_remove

<tf.Tensor: shape=(50000,), dtype=bool, numpy=array([False, False, False, ...,  True,  True,  True])>

In [101]:
a[..., :-1]

<tf.Tensor: shape=(49999,), dtype=bool, numpy=array([False, False, False, ...,  True,  True,  True])>

In [100]:
a = tf.math.cumsum(tf.nn.softmax(tf.sort(outputs[0], direction='DESCENDING'), axis=-1), axis=-1)>0.20
a

<tf.Tensor: shape=(50000,), dtype=bool, numpy=array([False, False, False, ...,  True,  True,  True])>

In [97]:
tf.math.cumsum(tf.nn.softmax(tf.sort(outputs[0], direction='DESCENDING'), axis=-1), axis=-1)

<tf.Tensor: shape=(50000,), dtype=float32, numpy=
array([0.069024  , 0.10334598, 0.1206357 , ..., 0.9999627 , 0.9999627 ,
       0.9999627 ], dtype=float32)>

In [96]:
tf.nn.softmax(tf.sort(outputs[0], direction='DESCENDING'), axis=-1)

<tf.Tensor: shape=(50000,), dtype=float32, numpy=
array([6.9024004e-02, 3.4321975e-02, 1.7289728e-02, ..., 2.3193357e-11,
       2.0838747e-11, 3.9051175e-12], dtype=float32)>

In [95]:
tf.argsort(outputs[0], direction='DESCENDING')

<tf.Tensor: shape=(50000,), dtype=int32, numpy=array([ 2592, 47441, 47453, ...,    73, 42568,     3])>

In [94]:
tf.sort(outputs[0], direction='DESCENDING')

<tf.Tensor: shape=(50000,), dtype=float32, numpy=
array([  8.840773 ,   8.142104 ,   7.456431 , ..., -12.973081 ,
       -13.0801325, -14.754661 ], dtype=float32)>

In [107]:
tf.random.categorical(tf.constant([a]), 1).numpy().tolist()[0]

[47453]

In [108]:
vocab.to_tokens(tf.random.categorical(tf.constant([a]), 1).numpy().tolist()[0])

['문에']

In [82]:
tf.random.categorical(tf.constant([a]), 1)

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[2592]], dtype=int64)>

In [92]:
[i for i in a.tolist() if i != -99999.]

[8.84077262878418, 8.142104148864746, 7.456430912017822]

In [81]:
tf.constant([a])

<tf.Tensor: shape=(1, 50000), dtype=float32, numpy=
array([[-99999., -99999., -99999., ..., -99999., -99999., -99999.]],
      dtype=float32)>

In [106]:
a = outputs[0].numpy()

a[outputs[0] < tf.math.top_k(outputs[0], 3)[0][..., -1, None]] = -99999
a

array([-99999., -99999., -99999., ..., -99999., -99999., -99999.],
      dtype=float32)

In [77]:
outputs[0].numpy()[outputs[0] < tf.math.top_k(outputs[0], 3)[0][..., -1, None]] = -99999

In [76]:
outputs[0].numpy()[outputs[0] < tf.math.top_k(outputs[0], 3)[0][..., -1, None]]

array([ -1.136622,   2.885709, -12.96938 , ...,  -6.681997,  -7.144026,
        -5.017031], dtype=float32)

In [71]:
outputs[0] < tf.math.top_k(outputs[0], 3)[0][..., -1, None]

<tf.Tensor: shape=(50000,), dtype=bool, numpy=array([ True,  True,  True, ...,  True,  True,  True])>

In [75]:
tf.math.top_k(outputs[0], 3)[0][..., -1]

<tf.Tensor: shape=(), dtype=float32, numpy=7.456431>

In [70]:
tf.math.top_k(outputs[0], 3)[0][..., -1, None]

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([7.456431], dtype=float32)>

In [69]:
tf.math.top_k(outputs[0], 3)[0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([8.840773, 8.142104, 7.456431], dtype=float32)>

In [67]:
outputs[0][2592]

<tf.Tensor: shape=(), dtype=float32, numpy=8.840773>

In [68]:
outputs[0][47441]

<tf.Tensor: shape=(), dtype=float32, numpy=8.142104>

In [66]:
tf.math.top_k(outputs[0], 3)

TopKV2(values=<tf.Tensor: shape=(3,), dtype=float32, numpy=array([8.840773, 8.142104, 7.456431], dtype=float32)>, indices=<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 2592, 47441, 47453])>)

In [61]:
outputs[0].shape

TensorShape([50000])

In [57]:
outputs[0]

<tf.Tensor: shape=(50000,), dtype=float32, numpy=
array([ -1.136622,   2.885709, -12.96938 , ...,  -6.681997,  -7.144026,
        -5.017031], dtype=float32)>

In [56]:
vocab.to_tokens(2592)

'문에'

In [55]:
vocab.to_tokens

<bound method Vocab.to_tokens of Vocab(size=50000, unk="<unk>", reserved="['<pad>', '<s>', '</s>']")>

In [49]:
tf.argmax(outputs)

<tf.Tensor: shape=(50000,), dtype=int64, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=int64)>

In [50]:
tf.argmax(outputs, axis = -1)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([2592], dtype=int64)>

In [52]:
tf.argmax(outputs, axis = -1).numpy()

array([2592], dtype=int64)

In [53]:
tf.argmax(outputs, axis = -1).numpy().tolist()

[2592]

In [54]:
tf.argmax(outputs, axis = -1).numpy().tolist()[0]

2592

In [47]:
gpt_model(tf.constant([0, 4499])[None, :])[:,-1,:]

<tf.Tensor: shape=(1, 50000), dtype=float32, numpy=
array([[ -1.136622,   2.885709, -12.96938 , ...,  -6.681997,  -7.144026,
         -5.017031]], dtype=float32)>

In [48]:
outputs = gpt_model(tf.constant([0, 4499])[None, :])[:,-1,:]

In [46]:
gpt_model(tf.constant([0, 4499])[None, :])

<tf.Tensor: shape=(1, 2, 50000), dtype=float32, numpy=
array([[[  2.5147848,   4.4742403, -10.7896185, ...,  -5.0172014,
          -6.3835497,  -5.6714473],
        [ -1.136622 ,   2.885709 , -12.96938  , ...,  -6.681997 ,
          -7.144026 ,  -5.017031 ]]], dtype=float32)>

In [44]:
tf.constant([0, 4499])

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([   0, 4499])>

In [45]:
tf.constant([0, 4499])[None, :]

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[   0, 4499]])>

In [40]:
vocab[['▁이때']]

[4499]

In [37]:
tf.constant([0,] + )

TypeError: can only concatenate list (not "int") to list

In [16]:
vocab['▁이때']

4499

In [18]:
vocab[vocab.bos_token]

0

In [19]:
vocab['<s>']

0

In [28]:
vocab['</s>']

1

In [29]:
vocab['<unk>']

5

In [30]:
vocab['<pad>']

3

In [31]:
vocab

Vocab(size=50000, unk="<unk>", reserved="['<pad>', '<s>', '</s>']")

In [24]:
type(vocab)

gluonnlp.vocab.bert.BERTVocab

In [20]:
vocab[:5]

TypeError: unhashable type: 'slice'

In [17]:
vocab.bos_token

'<s>'

In [13]:
vocab['이때']

5

In [12]:
tokenizer("이때")

['▁이때']

In [10]:
generate_sent("이때", gpt_model, greedy=True)

'이때문에 일부 전문가들은 “이번 사건은 ‘제2의 삼성’을 꿈꾸는 삼성의 내부 사정을 잘 보여주는 사례”라고 평가했다.'

In [11]:
generate_sent("이때", gpt_model, top_k=0, top_p=0.95) # Nucleus Sampling

'이때 JPLH사는 윈도우8프로세서 칩을 채용한 그래픽카드를 출시한다.'

In [112]:
generate_sent("이때", gpt_model, top_k=0, top_p=0.95) # Nucleus Sampling

'이때 그녀가 몇 걸음 다가서자 곧바로 대답이 나온다.'

In [113]:
generate_sent("이때", gpt_model, top_k=0, top_p=0.95) # Nucleus Sampling

"이때 '메리트폴리오'는 LA타임스, CNBC등 대부분의 경제지들이 단기경제연구소(RED)라고 주목하지만 단기경제연구소는 웰스파고, JP모간 등 일부 유력지들은 정부와 외부지원을 받는 민간인 연구소는 부정적이라며 경계하고 있다."

In [114]:
generate_sent("이때", gpt_model, top_k=0, top_p=0.20) # Nucleus Sampling

'이때 문 후보는 "안 후보가 단일화의 조건으로 정치쇄신을 내걸었는데, 안 후보가 이를 수용하지 않고 있다"며 "안 후보가 정치쇄신을 얘기하는데, 정치쇄신을 하려면 먼저 민주당이 먼저 쇄신하는 모습을 보여야 한다"고 말했다.'

In [115]:
generate_sent("이때", gpt_model, top_k=0, top_p=0.20) # Nucleus Sampling

'이때도 주군은 자신이 어떤 인물인지, 어떤 일을 하는지, 어떤 상황인지, 어떤 인물인지 알지 못한다.'

In [315]:
tf.random.categorical(tf.constant([[1.0,2.0,3.0,4.0,5.0]]), 3)

<tf.Tensor: shape=(1, 3), dtype=int64, numpy=array([[3, 4, 4]], dtype=int64)>

In [317]:
DATA_IN_PATH = './data_in/KOR/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='utf8').readlines()]
print(sents[:2])

['그때에 김첨지는 대수롭지 않은듯이,', '만일 김첨지가 주기를 띠지 않았던들 한 발을 대문에 들여놓았을 제 그곳을 지배하는 무시무시한 정적(靜寂) ― 폭풍우가 지나간 뒤의 바다 같은 정적이 다리가 떨렸으리라.']


In [321]:
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],] + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    # ex. ['<s>', '▁그때', '에', '▁김', '첨', '지는', 
    # '▁대수', '롭지', '▁않은', '듯이', ',', '</s>']
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])
    
# MAX_LEN : 30
input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token]) # vocab['<pad>']
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [322]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(\
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask
    acc = train_accuracy(real, pred)
    
    return tf.reduce_mean(acc)

In [323]:
gpt_model.compile(loss=loss_function,
                 optimizer=tf.keras.optimizers.Adam(1e-4),
                 metrics=[accuracy_function])

In [325]:
history = gpt_model.fit(input_data, output_data,
                       batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                       validation_split=0.1)

Epoch 1/10
16/16 [==============================] - 72s 3s/step - loss: 3.0223 - accuracy_function: 0.0993 - val_loss: 2.4907 - val_accuracy_function: 0.1127
Epoch 2/10
16/16 [==============================] - 52s 3s/step - loss: 2.5227 - accuracy_function: 0.1224 - val_loss: 2.3989 - val_accuracy_function: 0.1311
Epoch 3/10
16/16 [==============================] - 83s 5s/step - loss: 2.2738 - accuracy_function: 0.1391 - val_loss: 2.3849 - val_accuracy_function: 0.1453
Epoch 4/10
16/16 [==============================] - 84s 5s/step - loss: 2.0588 - accuracy_function: 0.1520 - val_loss: 2.3739 - val_accuracy_function: 0.1578
Epoch 5/10
16/16 [==============================] - 88s 6s/step - loss: 1.8566 - accuracy_function: 0.1654 - val_loss: 2.4125 - val_accuracy_function: 0.1699
Epoch 6/10
16/16 [==============================] - 85s 5s/step - loss: 1.6445 - accuracy_function: 0.1766 - val_loss: 2.4949 - val_accuracy_function: 0.1821
Epoch 7/10
16/16 [==============================] - 

In [324]:
tokenizer("그때에 김첨지는 대수롭지 않은듯이,")

['▁그때', '에', '▁김', '첨', '지는', '▁대수', '롭지', '▁않은', '듯이', ',']

In [320]:
MAX_LEN

30

In [318]:
vocab.padding_token

'<pad>'

In [319]:
vocab['<pad>']

3

In [326]:
DATA_OUT_PATH = './data_out/KOR'
model_name = "tf2_gpt2_finetuned_model"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)
    
gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out/KOR\tf2_gpt2_finetuned_model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [327]:
generate_sent('이때', loaded_gpt_model, greedy=True)

'이때에 마침 길가 선술집에서 김첨지는 김첨지의 등쌀에 넘어갈 제 여유조차 없었다.'

In [328]:
generate_sent('이때', gpt_model, greedy=True)

'이때에 마침 길가 선술집에서 김첨지는 김첨지의 등쌀에 넘어갈 제 여유조차 없었다.'

In [329]:
generate_sent('이때', loaded_gpt_model, top_k=0, top_p=0.95)

'이때에 오기가 좀 약한지 애걸복을 탄 듯 김첨지는 그 자리에 엎어져 눈을 크게 뜬 채 엉엉 울고 말았다.'

In [330]:
generate_sent('이때', loaded_gpt_model, top_k=0, top_p=0.95)

'이때문에 시장이 불안한 탓에 국고채 3년물 금리는 전날보다 0.04%포인트 오른 연 3.22%를 기록했다.'

In [331]:
generate_sent('이때', loaded_gpt_model, top_k=0, top_p=0.95)

'이때에 김첨지는 태자당을 뒤쫓아와 뜨개질 우는 가마니를 잡수시고 수염을 쓰다듬으며 너무 추워서 쓰러질 것 같았는데도 불구하고 아랑곳없이 화증을 내며 길바닥에 주저앉았다.'

In [332]:
generate_sent('이때', loaded_gpt_model, top_k=0, top_p=0.95)

'이때문에 박 대통령의 의중이 반영될 가능성이 없지 않아 있다.'

In [333]:
generate_sent('이때', loaded_gpt_model, top_k=0, top_p=0.20)

'이때문에 길가 선술집은 술렁이기 시작하였다.'

In [334]:
generate_sent('이때', loaded_gpt_model, top_k=0, top_p=0.20)

'이때에 이르러서는 그의 몸은 더욱 무거워졌고, 목은 더욱 무거워져 갔다.'

In [335]:
generate_sent('이때', loaded_gpt_model, top_k=0, top_p=0.20)

'이때문에 동광학교(東光學校)에는 전교생이 3천 명 남짓하였다.'

In [336]:
generate_sent('이때', loaded_gpt_model, top_k=0, top_p=0.20)

'이때에 이르러서는 마치 벌목꾼이 곡괭이를 들고 있는 양복쟁이를 끄는 것처럼 보였다.'